# Cross Validation for IDW Interpolation 
## Task 2A (IDW for continuous data only)

This document includes Python codes that conduct cross validation (CV) for Inverse Distance Weighting (IDW) Interpolation on water quality parameters, including 6 water quality parameters in arcpy environment:
- Dissolved oxygen (DO_mgl)
- Salinity (Sal_ppt)
- Turbidity (Turb_ntu)
- Temperature (T_c)
- Secchi (Secc_m)
- Total Nitrogen (TN_mgl) 

The analysis is conducted in the separate water bodies:
- Guana Tolomato Matanzas (GTM)
- Estero Bay (EB)
- Charlotte Harbor (CH)
- Biscayne Bay (BB)
- Big Bend Seagrasses (BBS)

**Tasks:**  

- **Task 2A Calculate the RMSE and Mean Error (ME) for IDW results using both continuous and discrete data**

- Task 2B Calculate the RMSE and Mean Error (ME) for IDW results using continuous data.

Time periods one year before and after storm event for Task 2A tests (seasons).
<br>
<div style="text-align: left;">
    <img src="../misc/TimePeriods.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

Summary of IDK and RK Accuracy Assessments.
<br>
<div style="text-align: left;">
    <img src="../misc/Table3.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

**Contents:**
* [1. Data Preprocess](#reg_preprocessing)
    * [1.1 Subsetting Dataset](#reg_subset)
    * [1.2 Preview Dataset](#reg_preview)
    * [1.3 Fill Unique ID](#reg_id)
* [2. Create Shapefile](#reg_create_shp)
* [3. Cross Validation for IDW](#reg_cv_idw)

# 1. Loading packages

In [1]:
import pandas as pd
import numpy  as np
import arcpy
from arcpy.sa import *
import os, time, math, importlib, sys
path = r'E:\Projects\SEACAR_WQ_2024\git\misc'
sys.path.insert(0, path)
import idw_rk
# !install conda install conda-forge::pyproj
import pyproj,csv

importlib.reload(idw_rk)

import warnings
warnings.filterwarnings('ignore')

# define scratch folder to avoid overwritting from parallel threats
arcpy.env.scratchWorkspace = r"E:\Projects\SEACAR_WQ_2024\scratch/IDW_con"

# 1. Data Preprocessing <a class="anchor" id="reg_preprocessing"></a>

## 1.1 Load csv files

In [2]:
gis_path = r'E:\Projects\SEACAR_WQ_2024/GIS_Data/'
dfCon = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Feb-21.csv', low_memory=False)

## 1.2 Subsetting Data <a class="anchor" id="reg_subset"></a>

### Selecting data from 9 am to 17 pm (daytime)

In [3]:
# Convert string to datetime
dfCon['SampleDate'] = pd.to_datetime(dfCon['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')


# Include date from 9:00 am to 17:00 pm
start_time = '08:00'
end_time = '18:00'

dfConTime = dfCon[dfCon['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]

## 1.3 Calculating average values at unique observation points

In [4]:
dfCon_Mean = dfConTime.groupby(['WaterBody','ParameterName','ParameterUnits', 'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym'])["ResultValue"].agg("mean").reset_index()
dfCon_Mean

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.796613
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.597323
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.309026
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.433010
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.499969
...,...,...,...,...,...,...,...,...,...
1820,Guana Tolomato Matanzas,Water Temperature,Degrees C,2022,Spring,30.050857,-81.367465,GTM,23.312820
1821,Guana Tolomato Matanzas,Water Temperature,Degrees C,2022,Winter,29.667071,-81.257403,GTM,19.372195
1822,Guana Tolomato Matanzas,Water Temperature,Degrees C,2022,Winter,29.737041,-81.245953,GTM,19.837073
1823,Guana Tolomato Matanzas,Water Temperature,Degrees C,2022,Winter,29.868851,-81.307428,GTM,20.105854


## 1.4 Convert coordinate system to EPSG: 3086

In [ ]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
dfCon_Mean[['x', 'y']] = dfCon_Mean.apply(transform_coordinates, axis=1)

## 2. Prepare for batch interpolation
### 2.1 Preset abbreviation for waterbody and parameter name

In [ ]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

### 2.2 Define the barrier files

In [ ]:
barrier_folder = os.path.join(gis_path, 'Barriers')
barrier_folder

barriers = []
for file in os.listdir(barrier_folder):
    if file.endswith(".shp"):
        barriers.append(os.path.join(barrier_folder, file))

for barrier in barriers:
    print(barrier)

### 2.3 Define waterbody boundary for spatial extent and masking

In [ ]:
waterbody_extent = os.path.join(gis_path, 'OEAT_Waterbody_Boundaries', 'OEAT_Waterbody_Boundary.shp')

unique_waterbodies = []
with arcpy.da.SearchCursor(waterbody_extent, ['WaterbodyA']) as cursor:
    for row in cursor:
        unique_waterbodies.append(row[0])

print("Unique Waterbodies:", unique_waterbodies)

### 2.4 Load the table of study periods,  parameters, and seasons

In [ ]:
seasons_con = pd.read_csv(gis_path + 'Seasons_con.csv', low_memory=False)

### 2.5 Define output folders### Define output folders

In [ ]:
shpCon_folder = gis_path + r"shapefiles_Con"
idwCon_folder = gis_path + r"idw_Con"

# Preview data
dfCon_Mean

## 2.6 Fill NaN RowID with unique ID, IDW function needs unique ID <a class="anchor" id="reg_id"></a>

In [ ]:
idw_rk.fill_nan_rowids(dfCon_Mean, 'RowID')
dfCon_Mean

# 3. Create Shapefiles <a class="anchor" id="reg_create_shp"></a># 2. Create Shapefile <a class="anchor" id="reg_create_shp"></a>

In [ ]:
# Empty the shapefile folder
idw_rk.delete_all_files(shpCon_folder)

# Merge interested with latitude and longitude columns
seasons_con_coord = idw_rk.merge_with_lat_long(seasons_con, dfCon_Mean)
seasons_con_coord

In [ ]:
idw_rk.create_shp_season(seasons_con_coord, shpCon_folder)

# 3. Cross Validation for IDW <a class="anchor" id="reg_cv_idw"></a>

In [ ]:
# Empty the shapefile folder
idw_rk.delete_all_files(idwCon_folder)

In [ ]:
# Select a section of table to process
seasons_slct = seasons_all.iloc[122:]

In [ ]:
# If the number of data points is less than 3，skipping calculate IDW
idw_rk.idw_interpolation(seasons_con, shpCon_folder, idwCon_folder, waterbody_extent, barrier_folder)